# <u>6 Important Fields to Be Extracted:</u>
* Applicant Name (basic)

* Loan Program (basic)

* Loan Amount (basic)

* Interest Rate (basic)

* Underwriting Fee (table-based)

* Appraisal Fee (table-based)





In [1]:
!pip install pymupdf
## if not already installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.9 MB/s eta 0:00:00


In [2]:
## upload file
from google.colab import files

uploaded = files.upload()

Saving lender-worksheet.pdf to lender-worksheet.pdf


In [ ]:
## inspect text
import fitz, re, json
doc = fitz.open('lender-worksheet.pdf')
text = ""
for page in doc:
    text += page.get_text("text")

print(text[:1000])


In [120]:
page = doc[0]
blocks = page.get_text("blocks") ## extract text in blocks

blocks.sort(key=lambda b: b[1])  # sort by y position

def get_nearby_value(label_keyword, blocks, indicator, y_tolerance=1):
    label_block = next((b for b in blocks if label_keyword.lower() in b[4].lower()), None) # find the next block that contains the label keyword
    if not label_block:
        return None
    label_y = label_block[1] # get the y0 of the matching block
    for b in blocks:
        if abs(b[1] - label_y) <= y_tolerance and indicator in b[4]:  # find block with similar y0
            return b[4]
    return None


In [121]:
def extractor(entry_title, possible_field_name, indicator, regex_pattern, reg_group=0):
  block = get_nearby_value(possible_field_name, blocks, indicator)
  if not block:
    return None
  value = re.search(regex_pattern, block).group(reg_group)

  location = page.search_for(value)

  rects = page.search_for(value)
  for r in rects:
      bbox = r

  return {
      entry_title: {
            "text": f"{entry_title}: {value}",
            "bbox": [round(bbox.x0, 2), round(bbox.y0, 2), round(bbox.x1, 2), round(bbox.y1, 2)] if bbox else None
        }
    }

applicant_name = extractor('Applicant Name', "Applicant", "",r"[A-Z][A-Za-z\.\']+(?: [A-Z][A-Za-z\.\']+)*(?: ?/ ?[A-Z][A-Za-z\.\']+(?: [A-Z][A-Za-z\.\']+)*)?"
)
loan_program = extractor('Loan Program', "Loan Program", "", r"\d{2}\s+[A-Z]+\s+[A-Z]+\s*-\s*[A-Za-z]+")

loan_amount = extractor('Loan Amount', "Loan Amount", "$", r"\$ ?[\d,]+")

interest_rate = extractor('Interest Rate', "Interest", "%", r"\d+\.\d+\s?%")

underwriting_fee = extractor('Underwriting Fee', "Underwriting", "$", r"([\d,]+\.\d{2})"
)
appraisal_fee = extractor('Appraisal Fee', "Appraisal", "$", r"Appraisal Fee[\s\S]*?\$[\s\n]*([\d,]+\.\d{2})", reg_group=1
)

In [122]:
fields = {}
for field in [underwriting_fee, appraisal_fee, applicant_name, loan_program, loan_amount, interest_rate]:
    if field:
        fields.update(field)

print(json.dumps(fields, indent=2))


{
  "Underwriting Fee": {
    "text": "Underwriting Fee: 550.00",
    "bbox": [
      501.12,
      194.36,
      525.61,
      203.29
    ]
  },
  "Appraisal Fee": {
    "text": "Appraisal Fee: 525.00",
    "bbox": [
      501.12,
      234.68,
      525.61,
      243.61
    ]
  },
  "Applicant Name": {
    "text": "Applicant Name: John Q. Smith / Mary A. Smith",
    "bbox": [
      100.8,
      78.44,
      215.49,
      87.37
    ]
  },
  "Loan Program": {
    "text": "Loan Program: 30 YEAR FIXED -Purchase",
    "bbox": [
      471.6,
      98.6,
      571.7,
      107.53
    ]
  },
  "Loan Amount": {
    "text": "Loan Amount: $ 380,000",
    "bbox": [
      116.64,
      161.96,
      152.29,
      170.89
    ]
  },
  "Interest Rate": {
    "text": "Interest Rate: 4.250 %",
    "bbox": [
      298.8,
      161.96,
      328.23,
      170.89
    ]
  }
}
